In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
torch.cuda.empty_cache()

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('font', serif='Arial') 
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'
random_seed = 420
torch.manual_seed(random_seed)

In [213]:
def uniform_initializer(out_dim, in_dim, cuda = True):
    tensor = torch.empty(out_dim, in_dim)
    if cuda:
        return torch.nn.init.uniform_(tensor, a=-2, b=2).cuda()
    else: 
        return torch.nn.init.uniform_(tensor, a=-2, b=2)
    
def apply_gaussian_noise(tensor, sd, device = torch.device("cuda:0")):
    tensor = tensor + (sd)*torch.randn(*tuple(tensor.shape)).to(device)
    return tensor

class simple_encoder_wthreshold():
    def __init__(self, out_dim, in_dim, epsilon, cuda = True):
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.W = uniform_initializer(out_dim, in_dim, cuda)
        self.device = torch.device("cuda:0") if cuda else torch.device("cpu")
        self.epsilon = epsilon
        
    def apply(self, X):
        return (torch.matmul(self.W, X) > self.epsilon).float()
    
    def apply_wnoise(self, X, sd):
        #print(X.shape, self.W.shape)
        if sd == 0:
            return self.apply(X)
        return (torch.matmul(apply_gaussian_noise(self.W, sd, device = self.device), X) > self.epsilon).float()
    
    def apply_wnoise_realistic(self, X, sd):
        #print(X.shape, self.W.shape)
        if sd == 0:
            return self.apply(X)
        encoded = torch.zeros((self.out_dim, X.shape[1])).to(self.device)
        for i in range(X.shape[1]):
            encoded[:,i] = (torch.matmul(apply_gaussian_noise(self.W, sd, device = self.device), X[:,i].view(-1,1)) > self.epsilon).float().view(-1)
        return encoded

In [214]:
in_dim = 40
out_dim =2000
vector = uniform_initializer(in_dim, 1, cuda = False)
encoder = simple_encoder_wthreshold(out_dim, in_dim, 0, cuda = False)
print(tensor)

tensor([[ 1.4663,  1.5043, -0.2077,  ...,  1.5154,  1.5975,  1.9062],
        [-1.9846, -1.4811,  1.0436,  ...,  1.7526,  1.6417, -0.9107],
        [ 1.7329, -1.1750,  0.6276,  ..., -1.8468,  0.7517,  1.2878],
        ...,
        [-0.3912,  1.6260, -0.7426,  ...,  1.0998,  0.6700,  1.3427],
        [ 1.3211,  1.9792, -0.3886,  ..., -1.2719,  0.3401,  0.4622],
        [ 0.1583, -0.7971, -1.7115,  ..., -0.3344, -1.9571,  1.5286]])


In [215]:
import time
total = 0
for i in range(100):
    tic = time.perf_counter()
    R = encoder.apply_wnoise(vector, sd=0.5)
    toc = time.perf_counter()
    total += (toc-tic)
print(total/100)

0.0006018169999879319
